In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly.offline import plot

#for offline plotting
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True) 

In [ ]:
google = pd.read_csv('GOOG.csv')  #2023-11-03,129.09,130.73,129.01,130.37,19517900 last row from excel (destination date)
google.head()
len(google)

1169

In [ ]:
google.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1169 entries, 0 to 1168
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   datetime  1169 non-null   object 
 1   open      1169 non-null   float64
 2   high      1169 non-null   float64
 3   low       1169 non-null   float64
 4   close     1169 non-null   float64
 5   volume    1169 non-null   int64  
dtypes: float64(4), int64(1), object(1)
memory usage: 54.9+ KB


In [ ]:
google['datetime'] = pd.to_datetime(google['datetime'])

In [ ]:
print(f'Dataframe contains stock prices between {google.datetime.min()} {google.datetime.max()}') 
print(f'Total days = {(google.datetime.max()  - google.datetime.min()).days} days')

Dataframe contains stock prices between 2019-03-18 00:00:00 2023-11-03 00:00:00
Total days = 1691 days


In [ ]:
google.describe()

,open,high,low,close,volume
count,1169.000000,1169.000000,1169.000000,1169.000000,1.169000e+03
mean,99.282361,100.446339,98.239470,99.362087,2.935388e+07
std,29.145560,29.413927,28.837961,29.110173,1.294590e+07
min,52.150000,52.370000,50.680000,51.810000,6.935060e+06
25%,71.810000,72.330000,71.060000,71.800000,2.112340e+07
50%,100.820000,102.550000,99.740000,101.280000,2.625652e+07
75%,125.130000,126.460000,124.150000,125.320000,3.323306e+07
max,151.860000,152.100000,149.890000,150.710000,1.241405e+08


In [ ]:
#google[['Open','High','Low','Close','Adj Close']].plot(kind='box')

In [ ]:
# Setting the layout for our plot
layout = go.Layout(
    title='Stock Prices of Alphabet (GOOG) from 2019.03.18 - 2023.11.02',
    xaxis=dict(
        title='Date',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='Price',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    )
)

google_data = [{'x':google['datetime'], 'y':google['close']}]
plot = go.Figure(data=google_data, layout=layout)

In [ ]:
#plot(plot) #plotting offline
iplot(plot)

In [ ]:
# Building the regression model
from sklearn.model_selection import train_test_split

#For preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

#For model evaluation
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score

In [ ]:
#Split the data into train and test sets
X = np.array(google.index).reshape(-1,1)
Y = google['close']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=101)

In [ ]:
# Feature scaling
scaler = StandardScaler().fit(X_train)

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
#Creating a linear model
lm = LinearRegression()
lm.fit(X_train, Y_train)

LinearRegression()

In [ ]:
#Plot actual and predicted values for train dataset
trace0 = go.Scatter(
    x = X_train.T[0],
    y = Y_train,
    mode = 'markers',
    name = 'Actual'
)
trace1 = go.Scatter(
    x = X_train.T[0],
    y = lm.predict(X_train).T,
    mode = 'lines',
    name = 'Predicted'
)
google_data = [trace0,trace1]
layout.xaxis.title.text = 'Day'
plot2 = go.Figure(data=google_data, layout=layout)

In [ ]:
iplot(plot2)

In [ ]:
#Calculate scores for model evaluation
scores = f'''
{'Metric'.ljust(10)}{'Train'.center(20)}{'Test'.center(20)}
{'r2_score'.ljust(10)}{r2_score(Y_train, lm.predict(X_train))}\t{r2_score(Y_test, lm.predict(X_test))}
{'MSE'.ljust(10)}{mse(Y_train, lm.predict(X_train))}\t{mse(Y_test, lm.predict(X_test))}
'''
print(scores)


Metric           Train                Test        
r2_score  0.5289005866736668	0.6012273925994092
MSE       399.23657043542505	332.9419827492228

